In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from helper import get_openai_api_key
KEY = get_openai_api_key()

In [ ]:
from openai import OpenAI

# Instantiate the client
client = OpenAI(
    api_key=KEY
)

In [ ]:
from pydantic import BaseModel
from enum import Enum
from typing import List, Optional, Literal
from openai import OpenAI
class UserPost(BaseModel):
    message: str

def make_post(output_class):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small 
                microblog-style post to them that sends some kind of 
                feedback, positive or negative.
            """},
            {"role": "user", "content": "Please write a post."},
        ],
        response_format=output_class,
    )
    return completion.choices[0].message.parsed

new_post = make_post(UserPost)
new_post

In [ ]:
class Mention(BaseModel):
    # The model chooses the product the mention is about,
    # as well as the social media post's sentiment
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']

    # Model can choose to respond to the user
    needs_response: bool
    response: Optional[str]

    # If a support ticket needs to be opened, 
    # the model can write a description for the
    # developers
    support_ticket_description: Optional[str]

In [ ]:
def analyze_mention(
    mention: str, 
    personality: str = "friendly"
) -> Mention:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"""
                Extract structured information from 
                social media mentions about our products.

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to 
                  inflammatory messages or bait.
                - A customized response to send to the user if we need 
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.
            """},
            {"role": "user", "content": mention},
        ],
        response_format=Mention,
    )
    return completion.choices[0].message.parsed

In [ ]:
post= analyze_mention(new_post.message)
post

In [ ]:
mention_json_string = post.model_dump_json(indent=2)
print(mention_json_string)